# 일급 함수 디자인 패턴

## 6.1 사례: 전략 패턴의 리팩토링

### 6.1.1 고전적인 전략

In [1]:
# 예제 6-1 플러그형 할인 전략을 가진 Order 클래스 구현
from abc import ABC, abstractmethod
from collections import namedtuple

Customer = namedtuple('Customer', 'name fidelity')

class LineItem:
    
    def __init__(self, product, quantity, price):
        self.product = product
        self.quantity = quantity
        self.price = price
        
    def total(self):
        return self.price * self.quantity
    
class Order: # 콘텍스트
    
    def __init__(self, customer, cart, promotion=None):
        self.customer = customer
        self.cart = list(cart)
        self.promotion = promotion
        
    def total(self):
        if not hasattr(self, '__total'):
            self.__total = sum(item.total() for item in self.cart)
        return self.__total

    def due(self):
        if self.promotion is None:
            discount = 0 
        else:
            discount = self.promotion.discount(self)
        return self.total() - discount
            
    def __repr__(self):
        fmt = '<Order total: {:.2f} due {:.2f}>'
        return fmt.format(self.total(), self.due())
    
class Promotion(ABC): # 전략: 추상 베이스 클래스
    
    @abstractmethod
    def discount(self, order):
        """할인액을 구체적인 숫자로 반환한다."""

class FidelityPromo(Promotion): # 첫 번째 구체적인 전략
    """충성도 포인트가 1000점 이상이 고객에게 전체 5% 할인 적용"""
    
    def discount(self, order):
        return order.total() * .05 if order.customer.fidelity >= 1000 else 0

class BulkItemPromo(Promotion): # 두 번째 구체적인 전략
    """20개 이상의 동일 상품을 구입하면 10% 할인 적용"""
    
    def discount(self, order):
        discount = 0
        for item in order.cart:
            if item.quantity >= 20:
                discount += item.total() * .1
        return discount 
    
class LargeOrderPromo(Promotion): # 세 번째 구체적인 전략
    """10종류 이상의 상품을 구입하면 전체 7% 할인 적용"""
    
    def discount(self, order):
        distinct_items = {item.product for item in order.cart}
        if len(distinct_items) >= 10:
            return order.total() * .07
        return 0

@abstract 데커레이터를 사용할 수 있도록 Promotion을 추상 베이스 클래스로 구현했다.

In [2]:
# 예제 6-2 여러 프로모션 할인을 적용해서 Order 클래스를 사용하는 예
joe = Customer('John Doe', 0) # joe의 춘성도 점수는 0점
ann = Customer('Ann Smith', 1100) # ann의 충성도 점수는 1100점
cart = [LineItem('banana', 4, .5), # 쇼핑 카트의 항목 종류가 3가지
       LineItem('apple', 10, 1.5),
       LineItem('watermelon', 5, 5.0)]
print(Order(joe, cart, FidelityPromo())) # joe에게 아무런 할인을 해주지 않음
print(Order(ann, cart, FidelityPromo())) # ann은 1000점이 넘으므로 5프로 할인

banana_cart = [LineItem('banana', 30, .5),
              LineItem('apple', 10, 1.5)]
print(Order(joe, banana_cart, BulkItemPromo())) # 30개를 구매한 바나나를 1.5 달러 할인받았다.

long_order = [LineItem(str(item_code), 1, 1.0)
              for item_code in range(10)]
print(Order(joe, long_order, LargeOrderPromo())) # 10종류 이상의 상품을 구매했기 때문에 7프로 할인 적용
print(Order(joe, cart, LargeOrderPromo()))

<Order total: 42.00 due 42.00>
<Order total: 42.00 due 39.90>
<Order total: 30.00 due 28.50>
<Order total: 10.00 due 9.30>
<Order total: 42.00 due 42.00>


### 6.1.2 함수지향 전략

In [3]:
# 예제 6-3 할인 전략을 함수로 구현한 Order 클래스
from collections import namedtuple

Customer = namedtuple('Customer', 'name fidelity')

class LineItem:
    
    def __init__(self, product, quantity, price):
        self.product = product
        self.quantity = quantity
        self.price = price
        
    def total(self):
        return self.price * self.quantity
    
class Order: # 콘텍스트
    
    def __init__(self, customer, cart, promotion=None):
        self.customer = customer
        self.cart = list(cart)
        self.promotion = promotion
        
    def total(self):
        if not hasattr(self, '__total'):
            self.__total = sum(item.total() for item in self.cart)
        return self.__total
    
    def due(self):
        if self.promotion is None:
            discount = 0
        else:
            discount = self.promotion(self) # 할인액을 계산하기 위해선 self.promotion()함수를 호출하면 된다.
        return self.total() - discount

    def __repr__(self):
        fmt = '<Order total: {:.2f} due: {:.2f}>'
        return fmt.format(self.total(), self.due())

# 추상 클래스가 제거되었다.
def fidelity_promo(order): # 각각의 구체적인 전략이 함수로 구현되었다.
    """충성도 포인트가 1000점 이상인 고객에게 전체 5% 할인 적용"""
    return order.total() * .05 if order.customer.fidelity >= 1000 else 0

def bulk_item_promo(order):
    """20개 이상의 동일 상품을 구입하면 10% 할인 적용"""
    discount = 0
    for item in order.cart:
        if item.quantity >= 20:
            discount += item.total() * .1
    return discount

def large_order_promo(order):
    """10종류 이상의 상품을 구입하면 전체 7% 할인 적용"""
    distinct_items = {item.product for item in order.cart}
    if len(distinct_items) >= 10:
        return order.total() * .07
    return 0

In [4]:
# 예제 6-4 할인 전략을 함수로 정의한 Order 클래스의 사용 예
joe = Customer('John Doe', 0)
ann = Customer('Ann Smith', 1100)
cart = [LineItem('banana', 4, .5),
       LineItem('apple', 10, 1.5),
       LineItem('watermelon', 5, 5.0)]
print(Order(joe, cart, fidelity_promo)) # 함수 인수를 인수로 전달
print(Order(ann, cart, fidelity_promo))

banana_cart = [LineItem('banana', 30, .5),
              LineItem('apple', 10, 1.5)]
print(Order(joe, banana_cart, bulk_item_promo))

long_order = [LineItem(str(item_code), 1, 1.0)
             for item_code in range(10)]
print(Order(joe, long_order, large_order_promo))
print(Order(joe, cart, large_order_promo))

<Order total: 42.00 due: 42.00>
<Order total: 42.00 due: 39.90>
<Order total: 30.00 due: 28.50>
<Order total: 10.00 due: 9.30>
<Order total: 42.00 due: 42.00>


### 6.1.3 최선의 전략 선택하기: 단순한 접근법

In [5]:
# 예제 6-6 함수 리스트를 반복해서 최대 할인액을 찾아내는 best_promo() 함수
promos = [fidelity_promo, bulk_item_promo, large_order_promo] # promos는 함수로 구현된 전략들의 리스트

def best_promo(order): # Order 객체를 인수로 받음
    """최대로 할인받을 금액을 반환한다."""
    return max(promo(order) for promo in promos) # 제너레이터 표현식을 이용해 최대 할인액 반환

In [6]:
# 예제 6-5 모든 할인을 적용해서 가장 큰 값을 반환하는 best_promo() 함수
print(Order(joe, long_order, best_promo)) # larger_order_promo 선택
print(Order(joe, banana_cart, best_promo)) # bulk_item_promo 선택
print(Order(ann, cart, best_promo)) # fidelity_promo 선택

<Order total: 10.00 due: 9.30>
<Order total: 30.00 due: 28.50>
<Order total: 42.00 due: 39.90>


[예제 6-6]은 가독성도 좋고 잘 작동하지만, 일부 코드가 중복되어 버그가 생길 여지가 있다.

### 6.1.4 모듈에서 전략 찾기

globals()를 사용한 경우

In [7]:
# 예제 6-7 모듈 전역 네임스페이스를 내부 조사해서 만든 promos 리스트
promos = [globals()[name] for name in globals() # globals() 함수가 반환한 딕셔너리에서 name을 반복
         if name.endswith('_promo') # _promo로 끝나는 name만 선택
         and name != 'best_promo'] # 무한 재귀 호출을 피하기 위해 자신은 걸러낸다.

def best_promo(order):
    """최대로 할인받을 금액을 반환한다."""
    return max(promo(order) for promo in promos) # best_promo 내부 구조는 유지

별도의 모듈을 만들고 best_promo()를 제외한 모든 프로모션 할인 함수를 그 모듈에 넣어, 적용할 수 있는 모든 할인 함수를 모으는 방법

In [8]:
# 예제 6-8 새로운 promotions 모듈을 내부 조사해서 만든 promos 리스트
import promotions, inspect # 주어진 주문에 대해 할인액을 계산하는 함수들만 promotions 모듈에 넣어야 함

promos = [func for name, func in 
         inspect.getmembers(promotions, inspect.isfunction)]

def best_promo(order):
    """최대로 할인받을 금액을 반환한다."""
    return max(promo(order) for promo in promos)

## 6.2 명령

In [9]:
# 예제 6-9 각각의 MacroCommand 객체는 내부에 명령 리스트를 갖고 있다.
class MacroCommand:
    """명령 리스트를 실행하는 명령"""
    
    # commands 인수로부터 리스트를 만들면 명령들이 반복 가능한 객체임이 보장되며, 각가의 MacroCommand 객체 안에 명령에 대한 참조를 복사
    def __init__(self, commands):
        self.commands = list(commands) 
    
    def __call__(self):
        for command in self.commands: # MacroCommand 객체가 호출되면 self.commands에 들어있는 명령이 순서대로 호출됨
            command()